## Nested Class
Class defined within a class is a nested class. A nested class can be:
1. Static: such class can only access static members of the enclosing class
2. Non static (also called inner class): can access every member of enclosing class (even private one)

In [ ]:
public class NestedClassDemo {

    public static void main(String[] args) {
        // Creating instance of StaticClass
        Outer.StaticClass staticClass = new Outer.StaticClass();

        // Creating instance of Inner Class
        Outer outer = new Outer();
        Outer.InnerClass innerClass1 = outer.new InnerClass();
        // ---or---
        Outer.InnerClass innerClass2 = new Outer().new InnerClass();
    }

}

class Outer{
    private int outerMember;
    private static int outerStaticMember;
    public String x = "Outer";

    // Static nested class
    public static class StaticClass{
        private int innerMember;

        public StaticClass(){
            // Can only access static members of the Outer class
            innerMember = outerStaticMember;
        }
    }

    // Non-static nested class
    public class InnerClass{
        private int innerMember;
        public String x = "Inner";

        // static variables inside inner class must be final
        public static final innerStatic = 25;

        public InnerClass() {
            innerMember = outerStaticMember;
        }

        // Shadowing
        public void printData(String x) {
            System.out.println("Local = " + x);
            System.out.println("Inner = " + this.x);
            System.out.println("Outer = " + Outer.this.x);
        }
    }
}

The reason why non static nested class can only have final static members can be illustrated by assuming the scenario where it is possible to create non-final static variables.

In [ ]:
class Outer{
    class Inner{
        public static int count = 0;
    }

    public static void main(String[] args){
        Outer o1 = new Outer();
        Outer o2 = new Outer();
        
        Outer.Inner i1 = o1.new Inner();
        Outer.Inner i2 = o2.new Inner();
        
        i1.count++;
        System.out.println(i2.count); // This would have printed 1
    }
}

An increment on a child of parent 1 causes a change in a child of a different parent, even though a Child class is attached to its Parent class, so children of different parents should have independent contexts.

**NOTE:** As part of `Record`s introduction in Java 16, inner classes can now have non-final static members. So the above restriction no longer applies.

**Static vs Non Static Member Class:** if we declare a member class that does not require access to an enclosing instance, always put the static modifier in its declaration. Otherwise each instance of this nested class will have a reference to the enclosing class instance which takes up space.

## Local Class
Local class is a variant of inner class. It is defined inside a method definition. Similar to inner classes, a local class can also access every member of enclosing class. It can also access local variables as long as they are `effectively final`.

In [ ]:
public class LocalClassDemo {
    private int outerMember;

    public void outerMethod() {
        int localVar = 5;

        // Local class can only have abstract or
        // final modifiers
        class Worker extends Thread {
            // Can't modify localVar here
            public void run() {
                System.out.println(localVar);
                System.out.println(outerMember);
            }
        }

        new Worker().start();
    }
}

Local class inside a static method can only access static variables of enclosing class. Also there is no `static` local class.

### Anonymous Class
Very similar to local class, and follows similar rules.

In [ ]:
// Anonymous class cannot have explicit constructor
addOnClickListener(new OnClickListener(){
    public onClick(Event e){
        // Implementation
    }
});

Coming to the question of local variables being effectively final, this can be explained by noticing the fact that the local variables in this case continue to exist even though the method has gone out of scope (closure). It would be odd if the variable could be altered by an area out of the inner class.

## Lambda Expression
A lambda expression is a simplified syntax for implementation of *functional interfaces*. Consider the snippet below:

In [ ]:
public class LambdaExpression {
    public static void main(String[] args) {
        LambdaExpression expr = new LambdaExpression();
        expr.addStateChangeListener(new StateChangeListener() {

            @Override
            public void onStateChange(String oldState, String newState) {
                // Implementation
            }
        });
    }

    public void addStateChangeListener(StateChangeListener sL) {
        String prev = "Previous State";
        String next = "Next State";
        sL.onStateChange(prev, next);
    }
}

@FunctionalInterface
interface StateChangeListener{
    public void onStateChange(String oldState, String newState);
}

The above code can be shortened to:

In [ ]:
public class LambdaExpression {
    public static void main(String[] args) {
        LambdaExpression expr = new LambdaExpression();
        expr.addStateChangeListener((x, y) -> {
            // Implementation
        });
    }

    public void addStateChangeListener(StateChangeListener sL) {
        String prev = "Previous State";
        String next = "Next State";
        sL.onStateChange(prev, next);
    }
}

@FunctionalInterface
interface StateChangeListener {
    public void onStateChange(String oldState, String newState);
}

The lambda expression syntax can take various forms as listed:
- `(x, y) -> { //Statements }`
- `(x, y) -> // One line implmentation`
- `x -> { //Statements }`
- `x -> // One line implmentation`
- `() -> { //Statements }`

Lambda expressions are basically objects, so we can assign it to variables
```java
StateChangeListener sL = (x, y) -> { /*Implementation*/ };
```

Lambda expression can only be used with functional interfaces. If an interface has 3 methods out of which 2 are default or static, then also it can be used in a lambda expression. 

`java.util.function` interface provides a number of different functional interfaces, each has a single method differing only in signature and return value:
- **Function:** takes a parameter of type T and returns value with type R. It is defined as:

In [ ]:
public interface Function<T, R> {
    R apply(T t);

    // ...
}

// Example usage:
Function<String, Integer> toInteger = (inputString) -> Integer.parseInt(inputString);

// Instance method in Function to chain functions
Function<String, Integer> toInteger = s -> Integer.parseInt(s);
Function<String, Integer> squareString = toInteger.andThen(i -> i * i);

- **UnaryOperator:** takes a single parameter and returns a parameter of the same type

In [ ]:
public interface UnaryOperator<T> extends Function<T, T> {
    static <T> UnaryOperator<T> identity() {
        return t -> t;
    }
}

// Example usage:
UnaryOperator<String> toUpper = s -> s.toUpperCase();

- **BiFunction**: takes a parameter of type T and a parameter of Type U and returns value with type R. It is defined as:

In [ ]:
public interface BiFunction<T, U, R> {
    R apply(T t, U u);

    // ...
}

// Example usage:
BiFunction<Integer, Integer, List<Integer>> fillRange = (start, end) -> {
    List<Integer> list = new ArrayList<>();
    for (int i = start; i <= end; i++) {
        list.add(i);
    }
    return list;
};

- **BinaryOperator:** takes a two parameters and returns a parameter of the same type

In [ ]:
public interface BinaryOperator<T> extends BiFunction<T,T,T> {
    // ....
}

// Example usage:
BinaryOperator<Integer, Integer> adder = (x, y) -> { return x + y};

- **Supplier:** represents a function that supplies a value of some sorts

In [ ]:
public interface Supplier<T> {
    T get();
}

// Example usage:
Supplier<Integer> random = () -> { return new Random().nextInt(); }

- **Consumer:**  represents an function that consumes a value without returning any value

In [ ]:
public interface Consumer<T> {
    void accept(T t);

    // ...
}

// Example usage
Consumer<Object> jsonPrint = o -> {
    ObjectMapper mapper = new ObjectMapper();
    try {
        System.out.println(mapper.writeValueAsString(o));
    } catch (JsonProcessingException e) {
        throw new RuntimeException(e);
    }
};

// Chaining other consumer
Consumer<Object> toString = jsonPrint.andThen(s -> System.out.println(s.toString()));

- **Predicate:** takes a single value as parameter, and returns true or false.

In [ ]:
public interface Predicate<T> {
    boolean test(T t);

    // ...
}

// Example usage
Predicate<Person> isRetired = p -> p.getAge() >= 60;

// Instance helper methods in Predicate
Predicate<Person> isNotRetired = isRetired.negate();
Predicate<Person> retiredAndMale = isRetired.and(p -> p.getGender().equals("M"));
Predicate<Person> retiredOrMale = isRetired.or(p -> p.getGender().equals("M"));

There are multiple other classes as well as primitive specialised versions like `LongFunction`, `DoubleSupplier`, etc.

### Method Reference

Is a compact way of representing a lambda expression that just calls as method, for example:

In [ ]:
List.of(1,2,3).stream().forEach(i -> System.out.println(i));
// Can be shortened to:
List.of(1,2,3).stream().forEach(System.out::println);

"Hello\nthere".lines().map(l -> l.toUpperCase());
// Can be shortened to:
"Hello\nthere".lines().map(String::toUpperCase);


List.of(List.of(1,2), List.of(3,4)).stream().map(e -> new HashSet<>(e));
// Can be shortened to:
List.of(List.of(1,2), List.of(3,4)).stream().map(HashSet::new);